[View in Colaboratory](https://colab.research.google.com/github/Jeevan-J/Python_Funcode/blob/Python/fault_categorization_using_cnn.ipynb)

In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive
!pip install -q keras
!pip install xlrd
!pip install xlsxwriter

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.
    100% |████████████████████████████████| 112kB 6.5MB/s 


In [0]:
import numpy as np
import re
import itertools
from collections import Counter
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import keras


def clean_str(string):
    """
    Tokenization/string cleaning for datasets.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()


def load_data_and_labels():
    """
    Loads polarity data from files, splits the data into words and generates labels.
    Returns split sentences and labels.
    """
    #Load data from xlsx
    df = pd.read_excel('drive/Colab Notebooks/Wuling TAC (technical assistant center)  case list.xlsx', sheet_name='信息反馈汇总');

    fault_labels = df['fault category']
    fault_labels = list(fault_labels[1:])
    fault_description = df['fault description (voice of customer)']
    fault_description = list(fault_description[1:])
    
    encoder = LabelEncoder()
    encoder.fit(fault_labels)
    no_of_faults = len(list(encoder.classes_))
    y = encoder.transform(fault_labels)
    y = keras.utils.to_categorical(y,num_classes=None)
    
    x_text = [s.strip() for s in fault_description]
    #x_text = [clean_str(sentence) for sentence in x_text]
    return [x_text, y, no_of_faults]


def pad_sentences(sentences, padding_word="P"):
    """
    Pads all sentences to the same length. The length is defined by the longest sentence.
    Returns padded sentences.
    """
    sequence_length = max(len(x) for x in sentences)
    padded_sentences = []
    for i in range(len(sentences)):
        sentence = str(sentences[i])
        num_padding = sequence_length - len(sentence)
        new_sentence = sentence + padding_word * num_padding
        padded_sentences.append(new_sentence)
    return padded_sentences


def build_vocab(sentences):
    """
    Builds a vocabulary mapping from word to index based on the sentences.
    Returns vocabulary mapping and inverse vocabulary mapping.
    """
    # Build vocabulary
    word_counts = Counter(itertools.chain(*sentences))
    # Mapping from index to word
    vocabulary_inv = [x[0] for x in word_counts.most_common()]
    vocabulary_inv = list(sorted(vocabulary_inv))
    # Mapping from word to index
    vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
    return [vocabulary, vocabulary_inv]


def build_input_data(sentences, labels, vocabulary):
    """
    Maps sentences and labels to vectors based on a vocabulary.
    """
    x = np.array([[vocabulary[word] for word in sentence] for sentence in sentences])
    y = np.array(labels)
    return [x, y]


def load_data():
    """
    Loads and preprocessed data for the dataset.
    Returns input vectors, labels, vocabulary, and inverse vocabulary.
    """
    # Load and preprocess data
    sentences, labels, output_classes = load_data_and_labels()
    #print(str(sentences[1]))
    sentences_padded = pad_sentences(sentences)
    vocabulary, vocabulary_inv = build_vocab(sentences_padded)
    x, y = build_input_data(sentences_padded, labels, vocabulary)
    #print(x)
    return [x, y, vocabulary, vocabulary_inv,output_classes]
  
def rebuild(encoded_labels):
    df = pd.read_excel('drive/Colab Notebooks/Wuling TAC (technical assistant center)  case list.xlsx', sheet_name='信息反馈汇总');

    fault_labels = df['fault category']
    fault_labels = list(fault_labels[1:])
    
    encoder = LabelEncoder()
    encoder.fit(fault_labels)
    fault_description = df['fault description (voice of customer)']
    fault_description = list(fault_description[1:])
    
    pred_labels = encoder.inverse_transform(encoded_labels)
    return pred_labels,fault_description

In [9]:
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam,SGD
from keras.models import Model
from sklearn.model_selection import train_test_split
#from data_helpers import load_data

print('Loading data')
x, y, vocabulary, vocabulary_inv, output_classes = load_data()
# x = x.reshape(len(x),len(x[1]))
# x.shape -> (10662, 56)
# y.shape -> (10662, 2)
# len(vocabulary) -> 18765
# len(vocabulary_inv) -> 18765

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# X_train.shape -> (8529, 56)
# y_train.shape -> (8529, 2)
# X_test.shape -> (2133, 56)
# y_test.shape -> (2133, 2)


sequence_length = len(x[1]) # 56
vocabulary_size = len(voucabulary_inv) # 18765
embedding_dim = 256
filter_sizes = [3,4,5]
num_filters = 512
drop = 0.5

epochs = 10
batch_size = 32

# this returns a tensor
print("Creating Model...")
inputs = Input(shape=(sequence_length,), dtype='int32')
embedding = Embedding(input_dim=vocabulary_size, output_dim=embedding_dim, input_length=sequence_length)(inputs)
reshape = Reshape((sequence_length,embedding_dim,1))(embedding)

conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)

maxpool_0 = MaxPool2D(pool_size=(sequence_length - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(conv_0)
maxpool_1 = MaxPool2D(pool_size=(sequence_length - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(conv_1)
maxpool_2 = MaxPool2D(pool_size=(sequence_length - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(conv_2)

concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
flatten = Flatten()(concatenated_tensor)
dropout = Dropout(drop)(flatten)
output = Dense(units=output_classes, activation='softmax')(dropout)

# this creates a model that includes
model = Model(inputs=inputs, outputs=output)

checkpoint = ModelCheckpoint('drive/Colab Notebooks/weights_sgd.{epoch:03d}-{val_acc:.4f}.hdf5', save_best_only=True, mode='auto')
#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(optimizer='RMSprop', loss='categorical_crossentropy', metrics=['accuracy'])
print("Traning Model...")
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, callbacks=[checkpoint], validation_data=(X_test, y_test))  # starts training
model.save_weights('weights.h5')

Loading data
Creating Model...
Traning Model...
Train on 30797 samples, validate on 7700 samples
Epoch 1/10
16544/30797 [===============>..............] - ETA: 3:03 - loss: 0.7726 - acc: 0.7171

30797/30797 [==============================] - 436s 14ms/step - loss: 0.7163 - acc: 0.7377 - val_loss: 0.6509 - val_acc: 0.7687
Epoch 2/10
 8832/30797 [=======>......................] - ETA: 4:42 - loss: 0.6018 - acc: 0.7746

30797/30797 [==============================] - 435s 14ms/step - loss: 0.6074 - acc: 0.7782 - val_loss: 0.6774 - val_acc: 0.7422
Epoch 3/10
 5920/30797 [====>.........................] - ETA: 5:18 - loss: 0.5318 - acc: 0.8061

30797/30797 [==============================] - 434s 14ms/step - loss: 0.5606 - acc: 0.7969 - val_loss: 0.6263 - val_acc: 0.7769
Epoch 4/10
 4800/30797 [===>..........................] - ETA: 5:33 - loss: 0.4919 - acc: 0.8246

30797/30797 [==============================] - 434s 14ms/step - loss: 0.5167 - acc: 0.8149 - val_loss: 0.6571 - val_acc: 0.7836
Epoch 5/10
 4384/30797 [===>..........................] - ETA: 5:36 - loss: 0.4474 - acc: 0.8410

...

30797/30797 [==============================] - 434s 14ms/step - loss: 0.4321 - acc: 0.8443 - val_loss: 0.6808 - val_acc: 0.7839
Epoch 7/10
 4160/30797 [===>..........................] - ETA: 5:41 - loss: 0.3508 - acc: 0.8788

30797/30797 [==============================] - 434s 14ms/step - loss: 0.3917 - acc: 0.8609 - val_loss: 0.7499 - val_acc: 0.7855
Epoch 8/10
 4128/30797 [===>..........................] - ETA: 5:42 - loss: 0.3106 - acc: 0.8917

30797/30797 [==============================] - 434s 14ms/step - loss: 0.3609 - acc: 0.8725 - val_loss: 0.7736 - val_acc: 0.7865
Epoch 9/10
 4128/30797 [===>..........................] - ETA: 5:41 - loss: 0.2663 - acc: 0.9048

30797/30797 [==============================] - 434s 14ms/step - loss: 0.3329 - acc: 0.8834 - val_loss: 0.8527 - val_acc: 0.7747
Epoch 10/10
 4096/30797 [==>...........................] - ETA: 5:41 - loss: 0.2847 - acc: 0.8962

30797/30797 [==============================] - 434s 14ms/step - loss: 0.3124 - acc: 0.8905 - val_loss: 0.8749 - val_acc: 0.7749


In [0]:
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam,SGD
from keras.models import Model
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import accuracy_score
#from data_helpers import load_data

print('Loading data')
x, y, vocabulary, vocabulary_inv, output_classes = load_data()


sequence_length = len(x[1]) # 56
vocabulary_size = len(vocabulary_inv) # 18765
embedding_dim = 256
filter_sizes = [3,4,5]
num_filters = 512
drop = 0.5

epochs = 10
batch_size = 32

# this returns a tensor
print("Creating Model...")
inputs = Input(shape=(sequence_length,), dtype='int32')
embedding = Embedding(input_dim=vocabulary_size, output_dim=embedding_dim, input_length=sequence_length)(inputs)
reshape = Reshape((sequence_length,embedding_dim,1))(embedding)

conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)

maxpool_0 = MaxPool2D(pool_size=(sequence_length - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(conv_0)
maxpool_1 = MaxPool2D(pool_size=(sequence_length - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(conv_1)
maxpool_2 = MaxPool2D(pool_size=(sequence_length - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(conv_2)

concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
flatten = Flatten()(concatenated_tensor)
dropout = Dropout(drop)(flatten)
output = Dense(units=output_classes, activation='softmax')(dropout)

# this creates a model that includes
model = Model(inputs=inputs, outputs=output)

model.compile(optimizer='RMSprop', loss='categorical_crossentropy', metrics=['accuracy'])
print("Predicting Model...")
model.load_weights('drive/Colab Notebooks/weights_sgd.003-0.7769.hdf5')
y_pred = np.rint(model.predict(x))
print(accuracy_score(y, y_pred))

y_pred1 = np.argmax(y_pred,axis=1)
Pred_fault_labels,fault_descriptions = rebuild(y_pred1)
writer = pd.ExcelWriter('drive/Colab Notebooks/output.xlsx', engine='xlsxwriter')
df1 = pd.DataFrame({'fault_descriptions': fault_descriptions})
df1.to_excel(writer, sheet_name='Sheet1')
df2 = pd.DataFrame({'Predicted_fault_labels': Pred_fault_labels})
df2.to_excel(writer, sheet_name='Sheet1', startcol = 4)
writer.save()